In [1]:
!pip install python-igraph jsonlines 

     |████████████████████████████████| 3.2MB 2.7MB/s 


In [0]:
import igraph
import jsonlines
import json
import numpy as np
from operator import itemgetter
from pandas import DataFrame
from linecache import getline
from subprocess import check_output
from collections import Counter

In [0]:
class Search():
  def __init__(self,search_file,user_file):

    self.search_details = search_file
    self.search_term_count = self.line_count(self.search_details) 
    self.search_terms = self._find_search_terms()

    self.user_details = user_file
    self.user_count = self.line_count(self.user_details)
    self.users = self._find_users()

    self.course_list = []
    self.course_list = self._find_courses()

    self.course_dict = {}
    self.course_dict = self._find_course_details('new_course_details.jsonl')
  
  def get_search_terms(self):
    return self.search_terms
  
  def get_users(self):
    return self.users

  def get_courses(self):
    return self.course_list

  def get_course_details(self):
    return self.course_dict
  
  def line_count(self,filename):
    return int(check_output(['wc', '-l', filename]).split()[0])

  def _find_course_details(self,course_details_file):
    with open(course_details_file) as file:
      for line in file:
        line = json.loads(line)
        self.course_dict[line['course_id']] = line['course_title']
    return self.course_dict

  def _find_courses(self):
    
    with open(self.search_details) as file:
      for line in file:
        line = json.loads(line)
        for course in line['course_clicked']:
          if course['course'] not in self.course_list:
            self.course_list.append(course['course'])
        for course in line['wishlist']:
          if course not in self.course_list:
            self.course_list.append(course)
        for course in line['enrollment']:
          if course not in self.course_list:
            self.course_list.append(course)
    return self.course_list

  def get_courses_of_user(self,user_id):
    user = int(np.where(self.users == user_id)[0]) + 1
    user_det = getline(self.user_details,user)
    user_det = json.loads(user_det)

    course_set = []

    if user_det['course_clicked']:
      for course in user_det['course_clicked']:
        if course['course'].strip(' ') not in course_set:
          course_set.append(course['course'].strip(' '))

    if user_det['course_enrollment']:
      for course in user_det['course_enrollment']:
        if course['course'].strip(' ') not in course_set:
          course_set.append(course['course'])

    if user_det['course_wishlist']:
      for c in user_det['course_wishlist']:
        if c.strip(' ') not in course_set:
          course_set.append(c.strip(' '))

    return course_set

  def _find_search_terms(self):
    search_terms = np.array([ '' for i in range(self.search_term_count)],dtype='object')
    i = 0
    with open(self.search_details) as file:
      for line in file:
        line = json.loads(line)
        search_terms[i] = line['search_term'][0]
        i+=1
    return search_terms
  
  def _find_users(self):
    users = np.array([ '' for i in range(self.user_count)],dtype='object')
    i = 0
    with open(self.user_details) as file:
      for line in file:
        line = json.loads(line)
        users[i] = line['user']
        i+=1
    return users

  def _calculate_similarity(self,clicks_x,clicks_y,wishlist_x,wishlist_y,enrollment_x,enrollment_y):
    def jaccard_similarity(product_list_1,product_list_2):
      s1 = set(product_list_1)
      s2 = set(product_list_2)
      if len(s1) == 0 or len(s2) == 0:
        return 0.0
      return float(len(s1.intersection(s2))/len(s1.union(s2)))

    return (jaccard_similarity(clicks_x,clicks_y) + jaccard_similarity(wishlist_x,wishlist_y) + jaccard_similarity(enrollment_x,enrollment_y))

  def get_edges(self):

    with open(self.search_details) as file_x:
      for line_x in file_x:
        query_x = json.loads(line_x)

        clicks_x = [ item_x['course'] for item_x in query_x['course_clicked'] ]
        wishlist_x = query_x['wishlist']
        enrollment_x = query_x['enrollment']

        with open(self.search_details) as file_y:
          for line_y in file_y:
            query_y = json.loads(line_y)

            clicks_y = [ item_y['course'] for item_y in query_y['course_clicked']]
            wishlist_y = query_y['wishlist']
            enrollment_y = query_y['enrollment']

            similarity = self._calculate_similarity(clicks_x,clicks_y,wishlist_x,wishlist_y,enrollment_x,enrollment_y)

            yield query_x['search_term'][0],query_y['search_term'][0],similarity

  def get_course_query_interest(self,course_id,query_term):

    line_no = int(np.where(self.search_terms == query_term)[0]) + 1
    query_line = getline(self.search_details,line_no)
    query_line = json.loads(query_line)

    clicks,clicks_sum = 0,0
    wish,wishlist_sum = 0,len(query_line['wishlist']) if query_line['wishlist'] else 0 
    enroll,enrollment_sum = 0,len(query_line['enrollment']) if query_line['enrollment'] else 0

    if query_line['course_clicked']:
      for course in query_line['course_clicked']:
        if course['course'] == course_id:
          clicks = course['count']
        clicks_sum += course['count']
    
    if query_line['wishlist']:
      if course_id in query_line['wishlist']:
        wish = 1
    
    if query_line['enrollment']:
      if course_id in query_line['enrollment']:
        enroll = 1

    a,b,c = 0,0,0

    if clicks_sum>0:
      a = clicks/clicks_sum

    if wishlist_sum > 0:
      b = wish/wishlist_sum

    if enrollment_sum > 0:
      c = enroll/enrollment_sum

    return a + b + c

  def get_course_user_interest(self,user_id,course_id):
    line_no = int(np.where(self.users == user_id)[0]) + 1
    query_line = getline(self.user_details,line_no)
    query_line = json.loads(query_line)

    clicks,clicks_sum = 0,0
    wish,wishlist_sum = 0,len(query_line['course_wishlist']) if query_line['course_wishlist'] else 0
    enroll,enrollment_sum = 0,len(query_line['course_enrollment']) if query_line['course_enrollment'] else 0

    if query_line['course_clicked']:
      for course in query_line['course_clicked']:
        if course['course'].strip(' ') == course_id:
          clicks = course['count']
        clicks_sum += course['count']

    if query_line['course_wishlist']:
      if course_id in query_line['course_wishlist']:
        wish = 1
    
    if query_line['course_enrollment']:
      for course in query_line['course_enrollment']:
        if course['course'].strip(' ') == course_id:
          enroll = 1
          break

    a,b,c = 0,0,0

    if clicks_sum>0:
      a = clicks/clicks_sum

    if wishlist_sum > 0:
      b = wish/wishlist_sum

    if enrollment_sum > 0:
      c = enroll/enrollment_sum

    return a + b + c

  def most_common_search(self,final_user_set):
    self.user_courses_map = {}
    for user in self.users:

      similar_users = final_user_set.get(user,[])
      similar_users_course_set = []
      final_course_set = []
      if not similar_users:
        continue
      for similar_user in similar_users:

        if user != similar_user:
          line_count = int(np.where(self.users == similar_user)[0]) + 1

          line = getline(self.user_details,line_count)
          line = json.loads(line)

          if line['course_clicked']:
            for course in line['course_clicked']:
              similar_users_course_set.append(course['course'])

          if line['course_enrollment']:
            for course in line['course_enrollment']:
              similar_users_course_set.append(course['course'])

          if line['course_wishlist']:
            for course in line['course_wishlist']:
              similar_users_course_set.append(course)

      course_subset = Counter(similar_users_course_set)
      for key,value in course_subset.items():
        if value>=4:
          final_course_set.append(key)
      self.user_courses_map[user] = final_course_set
    return self.user_courses_map

In [0]:
class SearchQueryGraph(object):

    """
      SearchQueryGraph contains methods for implementing the knowledge graph of search terms using the course clicks on a
      search term as historical data required for calculating the various metrics according to the research paper 
      Using Probabilistic Tag Modeling to Improve Recommendations Gokkaya et al August 2017 
      (http://ml4ed.cc/attachments/GokkayaUsing.pdf)
    """
    def __init__(self,query_list):

      self.tag_list=[] # List of Tags required for creating knowledge graph
      self.num_of_clusters = 0 
      self.query_list = query_list
      self.tag_search_query_map = {}
      self.graph = igraph.Graph(n=len(self.query_list),directed=True,vertex_attrs={'name':query_list},edge_attrs={'weight':[]})
      
      
    def create_graph(self, edge_list):
      """
        The data arguement takes two-tiered list with each inner list being of the form (node1,node2,weight).
        The _create_graph method creates a graph using igraph library and loads it into the object variable self.graph
      """
      # print(edge_list)
      self.graph.add_edge(edge_list[0],edge_list[1])
      # edges = self.graph.es['weight']
      # edges.extend(edgeweight)
      # print(edges)
      # self.graph.es['weight'] = x
      
    def add_weights(self,edgeweight):
      self.graph.es['weight'] = edgeweight    
        
    def create_clusters(self):
      """
        The _create_clusters method applies the walktrap graph community detection algorithm on the weighted graph 
        of the variable self.graph and creates the clusters obtained from the walktrap algritm on the object attribute 
        self.clusters
      """
      self.tag_list=[]  
      self.graph.layout_fruchterman_reingold()
      self.dendogram = self.graph.community_walktrap(weights=self.graph.es["weight"], steps = 1)
      self.num_of_clusters = self.dendogram.optimal_count
      self.clusters= self.dendogram.as_clustering()

    def _get_subgraph(self,cluster_id):
      """
        Returns subgraphs from the original graph according to the provided cluster_id
      """
      return self.clusters.subgraph(cluster_id)

    def _get_subgraphs(self):
      """
        Yields each subgraph of the graph containing a cluster one-by-one on each of the function call
      """
      for cluster_id in range(self.num_of_clusters):
        yield self.clusters.subgraph(cluster_id)

    def _get_tag_subgraph(self,subgraph,scores=False):
      """
        Returns the tag assigned to the given subgraph according to pagerank algorithm if scores arguement is set to false if set to False.
        
        Returns the tag assisgned to the given subgraph and the pagerank score for each node in the subgraph in the 
        format (node,score) if the scores arguement is set to True
      """
      #Applies pagerank algorithm on the given subgraph.
      pagerank_score = subgraph.pagerank()

      #Finds the tag with the maximum pagerank score in the given subgraph.
      tag_index = pagerank_score.index(max(pagerank_score))
      tag = subgraph.vs[tag_index]["name"]
      if(scores == False):
        return tag

      # Find the pagerank score of each node in the subgraph and assigned to cluster_score
      cluster_score=[]
      for index,node in enumerate(subgraph.vs):
        cluster_score.append([subgraph.vs[index]['name'],pagerank_score[index]])
      return tag, cluster_score

    def get_tags(self,scores=False):
      """
        Yields the tag assigned to each subgraph in the graph if scores is set to False.

        Yields the tag assigned to each subgraph and the pagerank score of each node in the subgraph if scores is set 
        to True
      """
      for subgraph in self._get_subgraphs():
        self.tag_search_query_map[self._get_tag_subgraph(subgraph)] = [item['name'] for item in subgraph.vs]
        yield self._get_tag_subgraph(subgraph,scores)

    def get_tag_list(self):

      """
        Returns list of tags assigned to each subgraph in the graph according to the pagerank score of each cluster
      """
      if not self.tag_list:
        for cluster_id in range(self.num_of_clusters):
          tag = self._get_tag_subgraph(self._get_subgraph(cluster_id))
          self.tag_list.append(tag)
      return self.tag_list

    def because_you_searched(self,search_term,course_tag_profile):
      out_list = []
      for tag,search_query_list in self.tag_search_query_map.items():
        if search_term not in search_query_list:
          continue
        else:
          course_tag_profile = course_tag_profile.sort_values(by=tag,axis=0,ascending=False)
          return course_tag_profile.index[course_tag_profile[tag]>0].tolist()

In [0]:
search_handler = Search('datasearchcomb.jsonl','datausercomblast.jsonl')
graph_handler = SearchQueryGraph(search_handler.get_search_terms())

In [0]:
# weights_arr = np.array([ 0 for i in range(search_handler.search_term_count**2) ],dtype='float')
w = []
count = search_handler.search_term_count**2
j=0
for i in search_handler.get_edges():
  if i[2] <0.2:
    continue
  graph_handler.create_graph(i[:2])
  w.append(i[2])
graph_handler.add_weights(w)

In [7]:
courses = search_handler.get_courses()
users = search_handler.get_users()
search_terms = search_handler.get_search_terms()
course_details = search_handler.get_course_details()
print(courses)
print(search_terms)

['608158782373810000420', '559158644019010000287', '757158798294310000420', '245156760451710000000', '261156788006210000000', '233158547303310000367', '748158780599110000419', '374158781517410000359', '16157907994810000048', '367158754185010000419', '170156784201510000081', '442158763935310000420', '777158696144210000417', '322157599163910000013', '22156405498210000048', '447158697470210000419', '470158701696410000420', '393158755559210000419', '838158696198810000419', '919158696284310000418', '280158237795110000289', '385156872993410000218', '234158410337210000287', '250158739892410000390', '759158666877610000390', '157158730481210000390', '585158780493310000390', '457158782807710000390', '949158609709610000390', '737158711625710000419', '705158582903510000302', '448157659010110000302', '515156638953610000000', '20158782281010000419', '494157805645010000294', '254158738215210000294', '162157622551210000289', '250158496964910000368', '118158497268010000368', '279157572608010000289', '2

In [0]:
graph_handler.create_clusters()
tags = graph_handler.get_tag_list()

In [18]:
tags

['the complete sql bootcamp 2020',
 'bigdata',
 'automation anywhere',
 'learn',
 'online yoga',
 'the art',
 'html',
 'robotic process automation',
 'soft skills',
 'digital marketing',
 'social',
 'mayuresh',
 'sql server administration part1',
 'kileshwar',
 'ios 13',
 'robotics',
 'data structure',
 'cyber security']

In [9]:
print(graph_handler.clusters)

Clustering with 38 elements and 18 clusters
[ 0] the complete sql bootcamp 2020, sql, complete design thinking
[ 1] bigdata, swapna
[ 2] vfx, finance, java, angular - the complete guide, automation anywhere,
     flutter, photoshop cc 2020, angular js, auto, complete guitar system,
     graphic design masterclass, mathematics, math, c
[ 3] learn flutter from scratch, learn basic calligraphy, learn
[ 4] online yoga
[ 5] the art
[ 6] html
[ 7] robotic process automation
[ 8] soft skills
[ 9] digital marketing
[10] social
[11] mayuresh, python for beginners, deep learning
[12] sql server administration part1
[13] kileshwar
[14] ios 13
[15] robotics
[16] data structure
[17] cyber security


In [10]:
course_tag_profile = DataFrame(index = courses,columns = tags)
for c in courses:
    #Lists of the probability of each cluster tag being assigned to each course
  print("*********************************************************")
  print("COURSE",c)
  for tag,querylist in graph_handler.get_tags(scores=True):
    prob = 0

    for query,score in querylist:    
        prob = prob + (search_handler.get_course_query_interest(c,query)*score)
    
    course_tag_profile[tag][c] = prob
    if(prob>0):
      print("TAG",tag,prob)

*********************************************************
COURSE 608158782373810000420
TAG the complete sql bootcamp 2020 0.4444444444444444
*********************************************************
COURSE 559158644019010000287
TAG the complete sql bootcamp 2020 0.08333333333333333
*********************************************************
COURSE 757158798294310000420
TAG the complete sql bootcamp 2020 0.25
TAG automation anywhere 0.07651186916458168
*********************************************************
COURSE 245156760451710000000
TAG the complete sql bootcamp 2020 0.1111111111111111
*********************************************************
COURSE 261156788006210000000
TAG the complete sql bootcamp 2020 0.1111111111111111
TAG sql server administration part1 1.0
*********************************************************
COURSE 233158547303310000367
TAG bigdata 0.6666666666666666
*********************************************************
COURSE 748158780599110000419
TAG automation anyw

In [11]:
import numpy as np
for c1 in courses:
  # Recomendation system for finding out the courses similar to a given course by finding out the inner product of 
  # courses 
  print("****************************************************")
  print("COURSE:c1",c1,course_details[c1])
  for c2 in courses:
    if np.sum(np.inner(np.array(course_tag_profile.loc[c1,:]),np.array(course_tag_profile.loc[c2,:])))>0:
      print("SIMILAR COURSES", c2,course_details[c2])
  print('\n')

****************************************************
COURSE:c1 608158782373810000420 The Complete SQL Bootcamp 2020
SIMILAR COURSES 608158782373810000420 The Complete SQL Bootcamp 2020
SIMILAR COURSES 559158644019010000287 Requirements Gathering Techniques for a Software Development (Copy0)
SIMILAR COURSES 757158798294310000420 The Complete Personal Finance Course
SIMILAR COURSES 245156760451710000000 MySQL
SIMILAR COURSES 261156788006210000000 SQL Server Administration Part1


****************************************************
COURSE:c1 559158644019010000287 Requirements Gathering Techniques for a Software Development (Copy0)
SIMILAR COURSES 608158782373810000420 The Complete SQL Bootcamp 2020
SIMILAR COURSES 559158644019010000287 Requirements Gathering Techniques for a Software Development (Copy0)
SIMILAR COURSES 757158798294310000420 The Complete Personal Finance Course
SIMILAR COURSES 245156760451710000000 MySQL
SIMILAR COURSES 261156788006210000000 SQL Server Administration Part

In [12]:
tag_user_structure = {}
user_tag_structure = {}
user_tag_profile = DataFrame(index = users,columns=tags)
for index,u in enumerate(users):
  print("*************************************************************")
  print("User",u,index)
  for tag,query_list in graph_handler.get_tags(scores=True):
    # print("Tag",tag)
    # print(query_list)
    prob_UT = []

    # Probablity of course being assigned a particular tag
    course_set = search_handler.get_courses_of_user(u)
    for c in course_set:
      # print(c)
      #Calculate the percentage of a user attention to the course
      prob_UC = search_handler.get_course_user_interest(u,c)
      # print(prob_UC)
      # Calculate the probabilty of the course being clicked after a search term
      prob = 0
      for query,score in query_list:
          w = search_handler.get_course_query_interest(c,query)
          prob = prob + (w*score)

      # print(prob)
      # Computing the probabilty of user being assigned the tag as sum of the above calculated values 
      prob_UTi = prob_UC * prob
      # print("P(C|T)",prob)
      if(prob_UTi>0):
          # print("P(U|T) for ",c, "course is = ",prob_UTi)
          prob_UT.append(prob_UTi)
    if(sum(prob_UT)):
      #Saving tag user structure
      user_arr = tag_user_structure.get(tag,[])
      user_arr.append(u)
      tag_user_structure[tag] = user_arr

      #Saving user tag structure
      tag_arr = user_tag_structure.get(u,[])
      tag_arr.append(tag)
      user_tag_structure[u] = tag_arr
      print("Tag",tag)
      print("P(U|T)=",sum(prob_UT))
      print('\n')
    x = sum(prob_UT)
    # print(x)
    user_tag_profile[tag][u] = x

*************************************************************
User 10000372 0
*************************************************************
User 10000386 1
*************************************************************
User 10000391 2
*************************************************************
User 10000282 3
Tag the complete sql bootcamp 2020
P(U|T)= 0.003584229390681003


Tag automation anywhere
P(U|T)= 0.07688477892002714


Tag learn
P(U|T)= 0.0689363964720964


Tag soft skills
P(U|T)= 0.03225806451612903


Tag mayuresh
P(U|T)= 0.00980392156862745


Tag kileshwar
P(U|T)= 0.005376344086021505


Tag ios 13
P(U|T)= 0.027777777777777776


Tag robotics
P(U|T)= 0.09108159392789374


Tag data structure
P(U|T)= 0.009259259259259259


*************************************************************
User 10000299 4
Tag automation anywhere
P(U|T)= 0.006994396066327421


Tag ios 13
P(U|T)= 0.014492753623188406


Tag robotics
P(U|T)= 0.09141583054626533


Tag cyber security
P(U|T)= 0.0144927536231

In [13]:
c_list = graph_handler.because_you_searched('sql',course_tag_profile)
for c in c_list:
   print("Course {} Name {}".format(c,course_details.get(c.strip(" "),"")))

Course 608158782373810000420 Name The Complete SQL Bootcamp 2020
Course 757158798294310000420 Name The Complete Personal Finance Course
Course 245156760451710000000 Name MySQL
Course 261156788006210000000 Name SQL Server Administration Part1
Course 559158644019010000287 Name Requirements Gathering Techniques for a Software Development (Copy0)


In [14]:
c_list = graph_handler.because_you_searched('social',course_tag_profile)
for c in c_list:
   print("Course {} Name {}".format(c,course_details.get(c.strip(" "),"")))

Course 470158701696410000420 Name Learn Social Psychology


In [15]:
final_user_structure = {}
final_user_set = {}
for u1 in users:
  # Recomendation system for finding out the courses similar to a given course by finding out the inner product of 
  # courses 
  print("****************************************************")
  print("User:u1",u1)
  u2_arr =[]
  user_set = []
  for u2 in users:
    user_score = np.sum(np.inner(np.array(user_tag_profile.loc[u1,:]),np.array(user_tag_profile.loc[u2,:])))
    if user_score>0.04:
      u2_arr.append([u2,user_score])
      user_set.append(u2)
      print("SIMILAR USERS", u2)
  print('\n')
  u2_arr = sorted(u2_arr,key = itemgetter(1),reverse=True)
  final_user_structure[u1] = u2_arr
  final_user_set[u1] = user_set

****************************************************
User:u1 10000372


****************************************************
User:u1 10000386


****************************************************
User:u1 10000391


****************************************************
User:u1 10000282
SIMILAR USERS 10000357
SIMILAR USERS 10000414
SIMILAR USERS 10000376
SIMILAR USERS 10000037
SIMILAR USERS 10000410
SIMILAR USERS 74454
SIMILAR USERS 526964
SIMILAR USERS 10000270
SIMILAR USERS 10000288
SIMILAR USERS 10000232
SIMILAR USERS 10000297


****************************************************
User:u1 10000299
SIMILAR USERS 10000270
SIMILAR USERS 10000288
SIMILAR USERS 10000232
SIMILAR USERS 10000297


****************************************************
User:u1 10000416
SIMILAR USERS 10000416
SIMILAR USERS 10000417
SIMILAR USERS 10000418
SIMILAR USERS 10000406
SIMILAR USERS 10000430
SIMILAR USERS 10000357
SIMILAR USERS 10000426
SIMILAR USERS 10000414
SIMILAR USERS 10000376
SIMILAR USERS 10000434


In [0]:
user_course_map = search_handler.most_common_search(final_user_set)

In [17]:
for user in user_course_map:
  print("*"*25)
  print("Users similar to {} searched these courses".format(user))
  user_course = user_course_map[user]
  if len(user_course) > 5:
    for i in range(5):
      print("Id: {} Name {}".format(user_course[i],course_details.get(user_course[i].strip(" "),'')))
      # print("Id: {} Name {}".format(user_course[i],course_details[user_course[i].strip(" ")]))
  else:
    for i in range(len(user_course)):
      print("Id: {} Name {}".format(user_course[i],course_details.get(user_course[i].strip(" "),'')))
  print('\n')

*************************
Users similar to 10000282 searched these courses
Id:  374158781517410000359 Name Learn flutter from scratch
Id: 374158781517410000359 Name Learn flutter from scratch
Id:  767158721964010000414 Name Jewellery Designing
Id:  226158859222410000420 Name 
Id: 515156638953610000000 Name Robotics


*************************
Users similar to 10000299 searched these courses
Id: 515156638953610000000 Name Robotics


*************************
Users similar to 10000416 searched these courses
Id:  777158696144210000417 Name Web Designing with HTML, CSS &amp; Bootstrap
Id:  257156994522510000218 Name Monthly meetings@CQA
Id:  340158201405910000218 Name Selenium WebDriver with Java
Id:  647158704803510000389 Name AI (level 1 )
Id:  31158437459810000357 Name Bootstrap


*************************
Users similar to 10000389 searched these courses
Id:  374158781517410000359 Name Learn flutter from scratch
Id: 374158781517410000359 Name Learn flutter from scratch
Id:  767158721964